# Data Quality Plan
Analysing the Dublin Bus Data provided to optimise model performance

## Summary
Each file has about 750,000 rows. Three quarters of a million.

## Ideas to handle data.
1. __Incremental Handing__
__Route Division__
Logic: Partition data into subsets of data which are internally comparable.
         Does it make sense that a single model should describe all routes?
         Since routes are different they should be modeled individually.

A machine can maintain 12 DF max in a single object. (atleast mine can)
If instead we extract each route from the data to build a dataframe of a single 
route. Then model it, take the best model. Now we have 100th of the data to handle in each increment.

When we have derived the model, we store it and its route, but can discard the data then. 

If we create a function that returns the model. We can either use re-assignment or scoping discard the dataframe in python.

For each route we store a model which is the sum of the models for the sections in which it is composed.

__Section Division__
Logic: The sum of the parts gives the value of the whole.

Many smaller data sets. This models the travel time for different sections in which the busses pass through. This reduces redundancy in the analysis of bus routes (which overlap)

Though since there are more sections than bus routes, there will be many more models to store, problem then becomes iterative. given t_0 the time we , we can calculate t_1 (the time we arrive in section1 after traversing seciont_0) then we apply the model for section 1 with t_1 as an input. (recirsive problem, easily described iteritavly)

the total time is the sum of all the predicted times for each section.

### Task
Create a function that tests different models and chooses the best.
Run this for each section storing each model. Then given a departure point and a destination we can derive the appropriate route and the sum of its sections.

How to derive the route. How often do time stamps occur. Accuracy of estimation of length of a section depends on the frequancy in which a time stamp is put out. Assume a section takes 100.5 seconds to cross. if a stamp is put out every second and there are 100 time stamps in a journey we know that the time taken is between 100 and 101 (small percentage error). The best estimate that can be made is that that time taken is in between 100 and 102.

If a time stamp occurs only every 50 seconds, then in a 100 second interval there could be 2 or 3 time stamps. i.e. we will predict it takes between 100 and 150.

Max-Min <= x <= (n+1/n)*(Max-Min), where n is the number of timestamps for a specific journey id occuring within a section)



##
2. __


We can use sparse data, take only every third entry for a given bus on a route. We then can fit the whole frame?


Best approach may be to develop models route by route, then save sparse data for increasing performance (if nexesary?)

SELECT t.id, t.key
FROM	
(
    SELECT id, key, ROW_NUMBER() OVER (ORDER BY key) AS rownum
    FROM datatable
) AS t
WHERE t.rownum % 30 = 0    -- or % 40 etc
ORDER BY t.key




In [1]:
#import sections
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from patsy import dmatrices
import matplotlib.patches as mpatches
import statsmodels.formula.api as sm
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from statsmodels.formula.api import logit
from sklearn.cross_validation import train_test_split

import os
import glob


C:\Users\Andy\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


def main():
    path = os.getcwd()
    contents = os.listdir(path)
    all_files = glob.glob(os.path.join(path,"*.csv.gz"))

    np_array_list = []
    for i in all_files:
        print(i)
    for file_ in all_files:
        df = pd.read_csv(file_,index_col=None, header=0)
        np_array_list.append(df.as_matrix())


    comb_np_array = np.vstack(np_array_list)
    big_frame = pd.DataFrame(comb_np_array)
    print("big frame defined")

    big_frame.head(1)

%timeit main()

%lprun -f main main()


#big_frame.columns = ["col1","col2 ...]

In [3]:
#method 1, ends at 12 dataframes.
path=os.getcwd() + "\\data"

def merge_frames(path):

    read = pd.read_csv
    concat = pd.concat
    
    contents = os.listdir(path)
    content_length = len(contents)
    
    accumulator = read(path+"\\"+contents[0], index_col=None, header=0)
    
    
    for i in range(1, content_length):
        print(i)
        next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
        accumulator = pd.concat([accumulator, next_df], axis=0)
    return accumulator

df_1 = merge_frames(path)

df_1.shape


1
2
3
4


C:\Users\Andy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


5
6
7
8
9
10
11
12


MemoryError: 